# 멋진 작사가 만들기

## 데이터 불러오기

In [1]:
# 모듈 import
import glob
import os
import re
import tensorflow as tf


txt_file_path = os.getenv('HOME')+'/aiffel/lyricist/data/lyrics/*'

txt_list = glob.glob(txt_file_path)

raw_corpus = []

# 여러개의 txt 파일을 raw_corpus에 담음
for txt_file in txt_list:
    with open(txt_file, "r") as f:
        raw = f.read().splitlines()
        raw_corpus.extend(raw)

print("데이터 크기:", len(raw_corpus))
#앞에서부터 4라인만 출력
print("Examples:\n", raw_corpus[:3])

데이터 크기: 187088
Examples:
 ["Now I've heard there was a secret chord", 'That David played, and it pleased the Lord', "But you don't really care for music, do you?"]


## 데이터 정제

In [2]:
for idx, sentence in enumerate(raw_corpus):
    if len(sentence) == 0: continue   # 길이가 0인 문장
    if sentence[-1] == ":": continue  # 문장의 끝이 : 인 문장

    if idx >15: break   # 문장 10개 확인
        
    print(sentence)

Now I've heard there was a secret chord
That David played, and it pleased the Lord
But you don't really care for music, do you?
It goes like this
The fourth, the fifth
The minor fall, the major lift
The baffled king composing Hallelujah Hallelujah
Hallelujah
Hallelujah
Hallelujah Your faith was strong but you needed proof
You saw her bathing on the roof
Her beauty and the moonlight overthrew her
She tied you
To a kitchen chair
She broke your throne, and she cut your hair
And from your lips she drew the Hallelujah Hallelujah


In [3]:

# 소문자로 바꾸고, 양쪽 공백 지움
# 특수문자 양쪽에 공백
# 여러개의 공백을 하나의 공백으로
# a-zA-Z?.!,¿가 아닌 문자를 하나의 공백으로
# 다시 양쪽 공백을 지웁니다
# 문장 시작에는 <start>, 끝에는 <end>를

def preprocess_sentence(sentence):
    sentence = sentence.lower().strip() # 1
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence) # 2
    sentence = re.sub(r'[" "]+', " ", sentence) # 3
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence) # 4
    sentence = sentence.strip() # 5
    sentence = '<start> ' + sentence + ' <end>' # 6
    return sentence

#어떻게 필터링되는지 확인
print(preprocess_sentence("This @_is ;;;sample        sentence."))

<start> this is sample sentence . <end>


In [4]:
#정제된 문장 모음
corpus = []

for sentence in raw_corpus:
    # 원하지 않는 문장은 넘김
    if len(sentence) == 0: continue
    if sentence[-1] == ":": continue
    if len(preprocess_sentence(sentence).split()) > 15: continue
    corpus.append(preprocess_sentence(sentence))
        
# 결과 10개만 확인
corpus[:10]

['<start> now i ve heard there was a secret chord <end>',
 '<start> that david played , and it pleased the lord <end>',
 '<start> but you don t really care for music , do you ? <end>',
 '<start> it goes like this <end>',
 '<start> the fourth , the fifth <end>',
 '<start> the minor fall , the major lift <end>',
 '<start> the baffled king composing hallelujah hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah <end>',
 '<start> hallelujah your faith was strong but you needed proof <end>']

In [5]:
# 토큰화 할 때Tokenizer와 pad_sequences를 사용

def tokenize(corpus):
    # 12000단어를 기억할 수 있는 tokenizer
    # 12000단어에 포함되지 못한 단어는 '<unk>'로 
    tokenizer = tf.keras.preprocessing.text.Tokenizer(
        num_words=12000, 
        filters=' ',
        oov_token="<unk>"
    )
    # corpus를 이용해 tokenizer 내부의 단어장을 완성
    tokenizer.fit_on_texts(corpus)
    # 준비한 tokenizer를 이용해 corpus를 Tensor로 변환
    tensor = tokenizer.texts_to_sequences(corpus)   
    # 시퀀스 길이를 일정하게 맞춤
    # 시퀀스가 짧다면 문장 뒤에 패딩을 붙여 길이를 맞춤
    # 패딩을 붙여 길이를 맞추고 싶다면 padding='pre'를 사용
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')  
    
    print(tensor,tokenizer)
    return tensor, tokenizer

tensor, tokenizer = tokenize(corpus)

[[   2   50    4 ...    0    0    0]
 [   2   15 2967 ...    0    0    0]
 [   2   33    7 ...   46    3    0]
 ...
 [   2    4  118 ...    0    0    0]
 [   2  258  194 ...   12    3    0]
 [   2    7   34 ...    0    0    0]] <keras_preprocessing.text.Tokenizer object at 0x7fe3ab03afd0>


In [6]:
print(tensor[:3, :10])

[[   2   50    4   95  303   62   53    9  946 6263]
 [   2   15 2967  871    5    8   11 5739    6  374]
 [   2   33    7   40   16  164  288   28  333    5]]


In [7]:
for idx in tokenizer.index_word:
    print(idx, ":", tokenizer.index_word[idx])

    if idx >= 10: break

1 : <unk>
2 : <start>
3 : <end>
4 : i
5 : ,
6 : the
7 : you
8 : and
9 : a
10 : to


In [8]:
# 마지막 토큰을 잘라내서 소스 문장을 생성
# 마지막 토큰은 <end>가 아니라 <pad>일 가능성이 높습니다.
src_input = tensor[:, :-1]  
# tensor에서 <start>를 잘라내서 타겟 문장을 생성
tgt_input = tensor[:, 1:]    

print(src_input[0])
print(tgt_input[0])

[   2   50    4   95  303   62   53    9  946 6263    3    0    0    0]
[  50    4   95  303   62   53    9  946 6263    3    0    0    0    0]


In [9]:
#훈련 데이터, 평가데이터 분리
#총 데이터의 20%는 평가 데이터로
from sklearn.model_selection import train_test_split
enc_train, enc_val, dec_train, dec_val = train_test_split(src_input, 
                                                          tgt_input,
                                                          test_size=0.2,
                                                          shuffle=True, 
                                                          random_state=34)

In [10]:
print('Source Train: ', enc_train.shape)
print('Target Train: ', dec_train.shape)

Source Train:  (124810, 14)
Target Train:  (124810, 14)


In [11]:
BUFFER_SIZE = len(src_input)
BATCH_SIZE = 256
steps_per_epoch = len(src_input) // BATCH_SIZE

VOCAB_SIZE = tokenizer.num_words + 1   

# 준비한 데이터 소스로부터 데이터셋을 만듭니다
dataset = tf.data.Dataset.from_tensor_slices((src_input, tgt_input))
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE, drop_remainder=True)
dataset

<BatchDataset shapes: ((256, 14), (256, 14)), types: (tf.int32, tf.int32)>

In [12]:
class TextGenerator(tf.keras.Model):
    def __init__(self, vocab_size, embedding_size, hidden_size):
        super().__init__()
        
        self.embedding = tf.keras.layers.Embedding(vocab_size, embedding_size)
        self.rnn_1 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.rnn_2 = tf.keras.layers.LSTM(hidden_size, return_sequences=True)
        self.linear = tf.keras.layers.Dense(vocab_size)
        
    def call(self, x):
        out = self.embedding(x)
        out = self.rnn_1(out)
        out = self.rnn_2(out)
        out = self.linear(out)
        
        return out
    
embedding_size = 256
hidden_size = 2048
model = TextGenerator(tokenizer.num_words + 1, embedding_size , hidden_size)

In [13]:
# 데이터셋에서 데이터 한 배치만 불러오는 방법
for src_sample, tgt_sample in dataset.take(1): break

model(src_sample)

<tf.Tensor: shape=(256, 14, 12001), dtype=float32, numpy=
array([[[-4.16395778e-07,  1.80750314e-04, -2.60765373e-04, ...,
          6.30866780e-05,  6.56587436e-05,  1.33362264e-04],
        [-8.11679711e-05,  2.15769847e-04, -6.42851403e-04, ...,
         -5.13171435e-05, -1.28693675e-04,  1.43707381e-04],
        [-2.11876453e-04,  2.73434416e-04, -6.53212483e-04, ...,
         -7.11624816e-05, -9.56941221e-05,  1.76204150e-04],
        ...,
        [-1.96870579e-03,  1.38334883e-03,  1.68865069e-03, ...,
          1.18336768e-03,  1.27355685e-04,  1.45303004e-03],
        [-2.38847919e-03,  1.49386167e-03,  1.99873606e-03, ...,
          1.52696995e-03,  2.01405914e-04,  1.65440096e-03],
        [-2.77968380e-03,  1.59160991e-03,  2.26885104e-03, ...,
          1.86440465e-03,  2.81908113e-04,  1.82204484e-03]],

       [[-4.16395778e-07,  1.80750314e-04, -2.60765373e-04, ...,
          6.30866780e-05,  6.56587436e-05,  1.33362264e-04],
        [-1.62962882e-04,  1.30684100e-04, -1

In [14]:
model.summary()

Model: "text_generator"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  3072256   
_________________________________________________________________
lstm (LSTM)                  multiple                  18882560  
_________________________________________________________________
lstm_1 (LSTM)                multiple                  33562624  
_________________________________________________________________
dense (Dense)                multiple                  24590049  
Total params: 80,107,489
Trainable params: 80,107,489
Non-trainable params: 0
_________________________________________________________________


In [15]:
#모델학습
optimizer = tf.keras.optimizers.Adam()
loss = tf.keras.losses.SparseCategoricalCrossentropy(
    from_logits=True,
    reduction='none'
)

model.compile(loss=loss, optimizer=optimizer)
model.fit(enc_train, dec_train, epochs=10, batch_size=256, validation_data = (enc_val, dec_val))

Epoch 1/10
488/488 [==============================] - 247s 501ms/step - loss: 3.4041 - val_loss: 3.0324
Epoch 2/10
488/488 [==============================] - 262s 537ms/step - loss: 2.8813 - val_loss: 2.8028
Epoch 3/10
488/488 [==============================] - 263s 540ms/step - loss: 2.6371 - val_loss: 2.6445
Epoch 4/10
488/488 [==============================] - 263s 539ms/step - loss: 2.4108 - val_loss: 2.5196
Epoch 5/10
488/488 [==============================] - 264s 541ms/step - loss: 2.1919 - val_loss: 2.4236
Epoch 6/10
488/488 [==============================] - 264s 541ms/step - loss: 1.9817 - val_loss: 2.3448
Epoch 7/10
488/488 [==============================] - 264s 541ms/step - loss: 1.7822 - val_loss: 2.2857
Epoch 8/10
488/488 [==============================] - 264s 541ms/step - loss: 1.5974 - val_loss: 2.2387
Epoch 9/10
488/488 [==============================] - 264s 542ms/step - loss: 1.4325 - val_loss: 2.2115
Epoch 10/10
488/488 [==============================] - 264s 541m

In [16]:
def generate_text(model, tokenizer, init_sentence="<start>", max_len=20):
    # 테스트를 위해서 입력받은 init_sentence도 텐서로 변환
    test_input = tokenizer.texts_to_sequences([init_sentence])
    test_tensor = tf.convert_to_tensor(test_input, dtype=tf.int64)
    end_token = tokenizer.word_index["<end>"]

    # 단어 하나씩 예측해 문장을 생성
    # 입력받은 문장의 텐서를 입력
    # 예측된 값 중 가장 높은 확률인 word index를 뽑아냅
    # 예측된 word index를 문장 뒤에 붙입니다
    # 모델이 <end>를 예측했거나, max_len에 도달했다면 문장 생성
    while True:
        # 1
        predict = model(test_tensor) 
        # 2
        predict_word = tf.argmax(tf.nn.softmax(predict, axis=-1), axis=-1)[:, -1] 
        # 3 
        test_tensor = tf.concat([test_tensor, tf.expand_dims(predict_word, axis=0)], axis=-1)
        # 4 max_len에 도달하지 않았다면, while 루프를 또 돌면서 다음 단어 예측
        if predict_word.numpy()[0] == end_token: break
        if test_tensor.shape[1] >= max_len: break

    generated = ""
    # tokenizer를 이용해 word index를 단어로 하나씩 변환 
    for word_index in test_tensor[0].numpy():
        generated += tokenizer.index_word[word_index] + " "

    return generated

In [18]:
test_sen = generate_text(model, tokenizer, init_sentence="<start> i love", max_len = 20)
print(test_sen)

<start> i love ma little nasty girl <end> 


# 회고

## 이번 프로젝트에서 어려웠던 점

### 모델 학습을 시키기 위해 epoch을 다 돌아가기까지 기다리는 시간이 너무 길었다. 이전의 프로젝트 들은 이렇게 오래도록 걸리지 않아서 몰랐지만 다른 분들께 들어보니 이 정도는  평범함 정도라고 하신다. epoch가 다 돌아간 후 Error가 뜨면 또 코드를 고치고 다시 돌리면 다시 기다려하는 것이 힘들었다. epoch 값을 10으로 맞추고 val_loss값을 2.2 이하로 맞추는 것이 너무 힘들었다. 

## 프로젝트를 진행하면서 알아내거나 모호한 점

### 모호한 점_ epoch를 계속 돌리다보면 val_loss 값이 떨어지다가 2,3 ~ 2.4 부분에서 다시 올라가는 이유에 대해 검색을 했을 때, 전처리 과정, 모델의 강제성, 학습 속도 등의 이유가 나왔다. 전처리 과정을 보았을 때는 잘 모르겠다. 아직 잘 모르는 부분이 많아서 그런 것 같다. 그리고 모델의 강제성 부분에서는 코드를 간결하게 하려고 해도 아직 내가 그 정도 수준이 되질 않아서 시도하지 못했다. 학습 속도는 epoch 값이 10이내 고정이라서 10으로 맞춰놓아서 아직 val_loss 값이 떨어지다가 다시 오르는 부분에 대한 나의 고민은 해결하지 못했다.

## 루브릭 평가 지표에 맞추기 위해 시도한 것

### epoch 값을 10으로 맞추고 val_loss값을 2.2 이하로 맞추기 위해 embedding size, hidden size 값을 계속 1024, 2048, 128, 256 계속 이렇게도 해보고 저렇게도 해보고 val_loss 값이 2.2로 맞춰지는가 싶으면 다시 올라가서 오버피팅이 아닌가 싶어서 다시 처음부터 코드 치고 size 바꾸고 다시 해보고 하다보니 해결!! 되었습니다.

## 자기다짐 

### 자기다짐....을 매번 적지만 항상 못지킨다.. 오늘의 자기다짐은 '뱉은 말을 지키자.'